Denne notebook gennemgår nogle af Python-biblioteket Pandas’ mange muligheder. Pandas er et bibliotek i Python, som man benytter til at arbejde med data, der er struktureret i rækker og kolonner. Man vil ofte vælge at strukturere sine data i rækker og kolonner, hvis man har samlet meget data, og der findes også mange datasæt, som allerede er struktureret således. Med lidt kendskab til Pandas er man godt hjulpet på vej til at kunne give sig i kast med dataanalyse af større datasæt.

Når du skal arbejde med Pandas bliver dette cheatsheet din ven: https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

Datasættet til denne notebook hedder 'ft_20151.zip' og ligger her https://github.com/KUBDatalab/datasets.

Når datasættet er downloadet, skal det 'unzippes'.

Det består af en csv-fil dels med af alle taler fra folketingets talerstol i 2015, dels med den metadata, der er knyttet til hver tale, f.eks. tid, navn, køn, parti, emne m.m. 


Datasættet er et 'subset' fra dette datasæt: Hansen, Dorte Haltrup, 2018, The Danish Parliament Corpus 2009 - 2017, v1, CLARIN-DK-UCPH Centre Repository, http://hdl.handle.net/20.500.12115/8.

In [ ]:
import pandas as pd

Hvis man allerede har data i en dataframe struktur, f.eks. i en csv fil, så kan man indlæse data på denne måde.

In [ ]:
df = pd.read_csv('.\\data\\ft_20151.csv')

Datasættet består dels af alle taler fra folketingets talerstol i 2015, dels af en data, der er knyttet til hver tale, f.eks. tid, navn, køn, parti, emne m.m.

Datasættet er et 'subset' fra dette datasæt: _Hansen, Dorte Haltrup, 2018, The Danish Parliament Corpus 2009 - 2017, v1, CLARIN-DK-UCPH Centre Repository, http://hdl.handle.net/20.500.12115/8._ 

Med .head(4) kan vi inspicere datasættets to første rækker.

In [ ]:
df.head(4)

.shape kan man bruge til at få vist datasættets omfang.

In [ ]:
df.shape

Det er klart, at der er en række interessante spørgsmål om danske politik, som man kan få svar på med
sådan et datasæt, og nedenfor skal vi prøve at finde svar på nogle af dem, og samtidigt opnå et kendskab til,
hvordan man bruger biblioteket Pandas.

## Sammentælling
Pandas har nogle funktioner der kan bruges til at sammentælle data.

En af funktionerne er .value_count(), der sammentæller alle forekomster af samme værdi.

F.eks. som nedenfor, hvor vi bruge .value_count() til at besvare spørgsmålet om, hvor ofte medlemmer fra de forskellige partier har taget ordet på folketingets talerstol.

In [ ]:
df["Party"].value_counts()

## Visualisering af sammentællinger
Pandas bliver ofte anvendt i kombination med visualiseringsbiblioteket Seaborn. Nedenfor er der et eksempel på et 'countplot'.

In [ ]:
import seaborn as sns
sns.countplot( y = df["Party"], data = df).set(title="Partier på talerstolen 2015" , xlabel='\nAntal', ylabel='Parti\n')

Lad os prøve samme kode igen, men nu vil vi besvare spørgsmålet om, hvilke emner er der blevet talt mest om i 2015?

In [ ]:
df["Subject 1"].value_counts()

In [ ]:
sns.countplot( y = df["Subject 1"], data = df).set(title="Emner", xlabel='\nAntal', ylabel='Parti\n')

## Lav et 'Subset' dvs. udvælg en eller flere kolonner

Man kan vælge flere kolonner med specifikke navne med df[['kol1', 'kol2', ... ]]. Læg mærke til at man skal
bruge to firkantede paranteser ved start og slut.

In [ ]:
df[['Party', 'Text']]

Man kan vælge en enkelt kolonne på denne måde.

In [ ]:
df['Party']

Man vil ofte have behov for at få mere data med ud end blot en kolonne, og derfor bruger man ofte denne
metode, hvor man vælger en værdi til at være lig med noget, f.eks. partinavnet skal være lig med 'V'.

In [ ]:
df2[df2['Party'] == 'V']

Man kan også få brug for at fremfinde bestemte vendinger eller ord, der indgår i en længere tekst.

Til dette kunne vi bruge df[df['Text'].str.contains('tekststreng')].

Vi kunne f.eks. filtere taler der indeholder tekststrengen 'islam'?

In [ ]:
new_df = df[df["Text"].str.contains("islam")]

Vi kan besvare spærgsmålet: Inden for hvilke emner, taler politikerne i folketinget om 'islam'? 

In [ ]:
sns.countplot( y = new_df ["Subject 1"], data = df).set(title="Emner", xlabel='\nAntal', ylabel='Parti\n')

# Regex

Nedenfor drejer det sig om at besvare spørgsmålet, hvilket parti taler mest om et ud valgt emne? I dette tilfælde ser vi på ordet 'islam'.

Vi benytter regulære udtryk (regex) til at holde styr på vores søgning. Regex er smart, fordi er udviklet til at fremfinde tekststrenge store datamængder. Regex er selvfølgelig lidt af en hovedpine, fordi det er ret besværligt at komme i gang med at sammesætte udtrykkene.

Vi kan bruge udtrykket '\b' til at ramme starten og slutningen af et ord ('\bislam\b'). På den måde er vi sikre på, at vi kun finder tekststrengen 'islam' og ikke tekststrenge, hvor 'islam' indgår i, f.eks. 'islamisk'.

Når vi er interesseret i at finde ordet 'islam' inklusiv alle endelser skal vi tilføej endelsen .+?\b efter islam.

In [ ]:
regex1 = r'\bislam\b'
regex2 = r'\bislam.+?\b'

In [ ]:
data_search = df[df['Text'].str.contains(regex1, regex=True)]

In [ ]:
sns.countplot( y = data_search['Party'], 
              data = data_search).set(title='Hvilket parti taler mest om emnet?', 
                                                                 xlabel='Antal af taler', ylabel='Parti')

# Filtrering af data ved brug af flere betingelser

Den boolske operator '&' kan anvendes til i de tilfælde, hvor der er brug for at flere betingelser er opfyldt. Der skal tilføjes nogle parenteser for at holde styr på de forskellige dele.

Vi vil gerne se nærmere på de taler.

In [ ]:
df['Role'].value_counts()

In [ ]:
new_df = df[(df['Role'] == 'medlem') & (df['Text'].str.contains(regex1, regex=True))]

In [ ]:
new_df 

In [ ]:
sns.countplot( y = new_df['Name'], hue='Party', data = new_df).set(title='Hvilket minister taler mest om emnet?', xlabel='Antal af taler', ylabel='Parti')

## Diakron frekvensanalyse

Hvornår taler man mest om noget?

Lav en tekststreng af alle taler.

In [ ]:
text_2015 = str(' '.join(df['Text'].str.lower()))

Split hvert år i 10 segmenter.

In [ ]:
interval = int(len(text_2015) / 10)

In [ ]:
segments = [text_2015[i:i + interval] for i in range(0, len(text_2015), interval)]
segments = segments[0:10]

In [ ]:
df2 = pd.DataFrame ({'Segment': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 
                   'Text': segments})

In [ ]:
# Søgeord
search_word = 'islam'
df2['search_word'] = search_word

In [ ]:
df2['Rel freq'] = df2['Text'].str.count(search_word) / df2['Text'].str.split().str.len().sum()

In [ ]:
sns.lineplot(data=df2, x='Segment', y="Rel freq", hue='search_word').set(title='Hvornår taler de mest om ' + search_word, xlabel='Tid', ylabel='Relativ frekvens')